## 1.导入手写数据集

电商18 180412126 王佳琦

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms

In [3]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example') #创建一个 ArgumentParser 对象。ArgumentParser 对象包含将命令行解析成 Python 数据类型所需的全部信息。
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')   #添加程序参数信息
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args(args=[])  #这里进行了修改，如果不修改，会报错。
args.cuda = not args.no_cuda and torch.cuda.is_available()
print(args)   #参数
 
#Sets the seed for generating random numbers. And returns a torch._C.Generator object.
torch.manual_seed(args.seed)   #随机数
if args.cuda:
    torch.cuda.manual_seed(args.seed)    

Namespace(batch_size=128, cuda=False, epochs=10, log_interval=10, no_cuda=False, seed=1)


In [4]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
trainset = datasets.MNIST('/data', train=True, download=True,transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=args.batch_size, shuffle=True, **kwargs)

In [5]:
testset= datasets.MNIST('/data', train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(
    testset,
    batch_size=args.batch_size, shuffle=True, **kwargs)
image, label = trainset[0]
print(len(trainset))
print(image.size())
image, label = testset[0]
print(len(testset))
print(image.size())

60000
torch.Size([1, 28, 28])
10000
torch.Size([1, 28, 28])


## 2. 定义GAN

In [6]:
import torch.autograd
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image
import os

In [7]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(784, 512),  # 输入特征数为784，输出为512
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),  # 进行非线性映射
            nn.Linear(512, 256),  # 进行一个线性映射
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()  # 也是一个激活函数，二分类问题中，
            # sigmoid可以班实数映射到【0,1】，作为概率值，
            # 多分类用softmax函数
        )

    def forward(self, x):
        x = self.dis(x)
        return x

定义判别器
将图片28x28展开成784，然后通过多层感知器，中间经过斜率设置为0.2的LeakyReLU激活函数，
最后接sigmoid激活函数得到一个0到1之间的概率进行二分类。

In [8]:
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(50, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.gen(x)
        return x

定义生成器
输入一个50维的0～1之间的高斯分布，然后通过第一层线性变换将其映射到256维,
然后通过LeakyReLU激活函数，接着进行一个线性变换，再经过一个LeakyReLU激活函数，
然后经过线性变换将其变成784维，最后经过Tanh激活函数是希望生成的假的图片数据分布
能够在-1～1之间。

In [9]:
D = discriminator()
G = generator()

In [10]:
criterion = nn.BCELoss()  # 是单目标二分类交叉熵函数
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

创建对象，判别器训练train
分为两部分：1、真的图像判别为真；2、假的图像判别为假
此过程中，生成器参数不断更新
首先需要定义loss的度量方式  （二分类的交叉熵）
其次定义 优化函数,优化函数的学习率为0.0003

## 3. 训练

In [12]:
dataloader=train_loader

In [13]:
if not os.path.exists('./img2'):
    os.mkdir('./img2')

In [14]:
def to_img(x):
    out = 0.5 * (x + 1)
    out = out.clamp(0, 1)  # Clamp函数可以将随机变化的数值限制在一个给定的区间[min, max]内：
    out = out.view(-1, 1, 28, 28)  # view()函数作用是将一个多行的Tensor,拼接成一行
    return out

In [15]:
batch_size = 128
num_epoch = 10
z_dimension = 50

In [23]:
mnist = datasets.MNIST(
    root='/data', train=True, transform=img_transform, download=False
)
# data loader 数据载入
dataloader = torch.utils.data.DataLoader(
    dataset=mnist, batch_size=batch_size, shuffle=True
)

In [24]:
device="cpu"

In [ ]:
for epoch in range(num_epoch):  # 进行多个epoch的训练
    for i, (img, _) in enumerate(dataloader):
        num_img = img.size(0)
        # view()函数作用是将一个多行的Tensor,拼接成一行
        # 第一个参数是要拼接的tensor,第二个参数是-1
        # =============================训练判别器==================
        img = img.view(num_img, -1)  # 将图片展开为28*28=784
        real_img = Variable(img) # 将tensor变成Variable放入计算图中
        real_label = Variable(torch.ones(num_img)) # 定义真实的图片label为1
        fake_label = Variable(torch.zeros(num_img))  # 定义假的图片的label为0
        # 计算真实图片的损失
        real_out = D(real_img)  # 将真实图片放入判别器中
        real_out=real_out.squeeze(-1)
        d_loss_real = criterion(real_out, real_label)  # 得到真实图片的loss
       
        real_scores = real_out  # 得到真实图片的判别值，输出的值越接近1越好
        # 计算假的图片的损失
        z = Variable(torch.randn(num_img, z_dimension)) # 随机生成一些噪声
        fake_img = G(z)  # 随机噪声放入生成网络中，生成一张假的图片
        fake_out = D(fake_img)  # 判别器判断假的图片
        fake_out=fake_out.squeeze(-1)
        d_loss_fake = criterion(fake_out, fake_label)  # 得到假的图片的loss
        fake_scores = fake_out  # 得到假图片的判别值，对于判别器来说，假图片的损失越接近0越好
        # 损失函数和优化
        d_loss = d_loss_real + d_loss_fake  # 损失包括判真损失和判假损失
        d_optimizer.zero_grad()  # 在反向传播之前，先将梯度归0
        d_loss.backward()  # 将误差反向传播
        d_optimizer.step()  # 更新参数
        # ==================训练生成器============================
        ################################生成网络的训练###############################
        # 原理：目的是希望生成的假的图片被判别器判断为真的图片，
        # 在此过程中，将判别器固定，将假的图片传入判别器的结果与真实的label对应，
        # 反向传播更新的参数是生成网络里面的参数，
        # 这样可以通过更新生成网络里面的参数，来训练网络，使得生成的图片让判别器以为是真的
        # 这样就达到了对抗的目的
        # 计算假的图片的损失
        z = Variable(torch.randn(num_img, z_dimension))  # 得到随机噪声
        fake_img = G(z)  # 随机噪声输入到生成器中，得到一副假的图片
        #
        output = D(fake_img)  # 经过判别器得到的结果
        output=output.squeeze(-1)
        g_loss = criterion(output, real_label)  # 得到的假的图片与真实的图片的label的loss
        # bp and optimize
        g_optimizer.zero_grad()  # 梯度归0
        g_loss.backward()  # 进行反向传播
        g_optimizer.step()  # .step()一般用在反向传播后面,用于更新生成网络的参数
        # 打印中间的损失
        if (i + 1) % 100 == 0:
            print('Epoch[{}/{}],d_loss:{:.6f},g_loss:{:.6f} '
                  'D real: {:.6f},D fake: {:.6f}'.format(
                epoch, num_epoch, d_loss.data.item(), g_loss.data.item(),
                real_scores.data.mean(), fake_scores.data.mean()  # 打印的是真实图片的损失均值
            ))
        if epoch == 0 and i==len(dataloader)-1:
            real_images = to_img(real_img.data)
            save_image(real_images, './img2/real_images.png')
        if i==len(dataloader)-1:
            fake_images = to_img(fake_img.data)
            save_image(fake_images, './img2/fake_images-{}.png'.format(epoch + 1))

Epoch[0/1000],d_loss:1.073028,g_loss:0.899909 D real: 0.586423,D fake: 0.411212
Epoch[0/1000],d_loss:0.896320,g_loss:1.149202 D real: 0.631438,D fake: 0.339410
Epoch[0/1000],d_loss:0.734944,g_loss:1.335426 D real: 0.692422,D fake: 0.295488
Epoch[0/1000],d_loss:0.649620,g_loss:1.433854 D real: 0.740392,D fake: 0.273143
Epoch[1/1000],d_loss:0.435950,g_loss:1.803776 D real: 0.815686,D fake: 0.186103
Epoch[1/1000],d_loss:0.353019,g_loss:2.500619 D real: 0.811870,D fake: 0.122193
Epoch[1/1000],d_loss:0.308498,g_loss:2.616590 D real: 0.849846,D fake: 0.125602
Epoch[1/1000],d_loss:0.363972,g_loss:2.501818 D real: 0.809835,D fake: 0.120675
Epoch[2/1000],d_loss:0.182097,g_loss:3.063644 D real: 0.932095,D fake: 0.096387
Epoch[2/1000],d_loss:0.218856,g_loss:2.843108 D real: 0.879663,D fake: 0.079066
Epoch[2/1000],d_loss:0.196825,g_loss:3.194733 D real: 0.921721,D fake: 0.104656
Epoch[2/1000],d_loss:0.184379,g_loss:3.068229 D real: 0.933476,D fake: 0.076010
Epoch[3/1000],d_loss:0.286171,g_loss:3.2

Epoch[25/1000],d_loss:0.298931,g_loss:2.742664 D real: 0.917156,D fake: 0.156689
Epoch[25/1000],d_loss:0.271076,g_loss:2.723765 D real: 0.892177,D fake: 0.110187
Epoch[26/1000],d_loss:0.144331,g_loss:3.031694 D real: 0.930855,D fake: 0.057061
Epoch[26/1000],d_loss:0.461756,g_loss:2.446883 D real: 0.906004,D fake: 0.222444
Epoch[26/1000],d_loss:0.254824,g_loss:2.607218 D real: 0.925242,D fake: 0.136547
Epoch[26/1000],d_loss:0.326400,g_loss:2.669156 D real: 0.889231,D fake: 0.153205
Epoch[27/1000],d_loss:0.233067,g_loss:3.094829 D real: 0.937383,D fake: 0.115993
Epoch[27/1000],d_loss:0.317012,g_loss:2.787318 D real: 0.908832,D fake: 0.163420
Epoch[27/1000],d_loss:0.549626,g_loss:2.343920 D real: 0.798121,D fake: 0.173611
Epoch[27/1000],d_loss:0.217946,g_loss:2.680912 D real: 0.892758,D fake: 0.073825
Epoch[28/1000],d_loss:0.323011,g_loss:2.207275 D real: 0.874593,D fake: 0.119247
Epoch[28/1000],d_loss:0.528061,g_loss:3.243329 D real: 0.794315,D fake: 0.139636
Epoch[28/1000],d_loss:0.3112

Epoch[51/1000],d_loss:0.522835,g_loss:2.414025 D real: 0.774140,D fake: 0.133656
Epoch[51/1000],d_loss:0.387238,g_loss:2.666146 D real: 0.872899,D fake: 0.156517
Epoch[51/1000],d_loss:0.539337,g_loss:2.988276 D real: 0.835287,D fake: 0.226963
Epoch[51/1000],d_loss:0.358740,g_loss:2.448452 D real: 0.917791,D fake: 0.179090
Epoch[52/1000],d_loss:0.457282,g_loss:2.376966 D real: 0.842398,D fake: 0.168274
Epoch[52/1000],d_loss:0.498213,g_loss:2.530876 D real: 0.755028,D fake: 0.107873
Epoch[52/1000],d_loss:0.309142,g_loss:2.694367 D real: 0.896470,D fake: 0.140469
Epoch[52/1000],d_loss:0.328072,g_loss:2.544828 D real: 0.901044,D fake: 0.153182
Epoch[53/1000],d_loss:0.281213,g_loss:2.776715 D real: 0.909998,D fake: 0.122859
Epoch[53/1000],d_loss:0.275485,g_loss:2.031130 D real: 0.902937,D fake: 0.126548
Epoch[53/1000],d_loss:0.955136,g_loss:2.431684 D real: 0.660526,D fake: 0.169679
Epoch[53/1000],d_loss:0.445237,g_loss:3.041344 D real: 0.811381,D fake: 0.145729
Epoch[54/1000],d_loss:0.2262

Epoch[76/1000],d_loss:0.733479,g_loss:1.939189 D real: 0.761885,D fake: 0.279928
Epoch[76/1000],d_loss:0.574234,g_loss:1.691353 D real: 0.797887,D fake: 0.213885
Epoch[77/1000],d_loss:0.600042,g_loss:1.723906 D real: 0.797972,D fake: 0.234402
Epoch[77/1000],d_loss:0.613214,g_loss:2.111619 D real: 0.776182,D fake: 0.218226
Epoch[77/1000],d_loss:0.893875,g_loss:1.978553 D real: 0.731708,D fake: 0.296866
Epoch[77/1000],d_loss:0.493109,g_loss:2.117611 D real: 0.889676,D fake: 0.260525
Epoch[78/1000],d_loss:0.591435,g_loss:2.562973 D real: 0.793889,D fake: 0.235512
Epoch[78/1000],d_loss:0.755149,g_loss:2.125910 D real: 0.749466,D fake: 0.256628
Epoch[78/1000],d_loss:0.666154,g_loss:1.842991 D real: 0.806907,D fake: 0.261317
Epoch[78/1000],d_loss:0.753538,g_loss:1.793078 D real: 0.766773,D fake: 0.247202
Epoch[79/1000],d_loss:1.126862,g_loss:1.716978 D real: 0.518631,D fake: 0.191975
Epoch[79/1000],d_loss:0.744941,g_loss:1.844459 D real: 0.737354,D fake: 0.229894
Epoch[79/1000],d_loss:0.7089

Epoch[102/1000],d_loss:0.795362,g_loss:1.934164 D real: 0.666149,D fake: 0.210161
Epoch[102/1000],d_loss:0.828266,g_loss:1.794080 D real: 0.667549,D fake: 0.251696
Epoch[102/1000],d_loss:0.822767,g_loss:1.605457 D real: 0.711255,D fake: 0.286736
Epoch[102/1000],d_loss:0.973801,g_loss:1.771258 D real: 0.671128,D fake: 0.292044
Epoch[103/1000],d_loss:0.925347,g_loss:1.840539 D real: 0.722470,D fake: 0.317415
Epoch[103/1000],d_loss:0.920751,g_loss:1.653916 D real: 0.642711,D fake: 0.284731
Epoch[103/1000],d_loss:0.954278,g_loss:1.838881 D real: 0.660079,D fake: 0.306989
Epoch[103/1000],d_loss:1.082367,g_loss:1.530589 D real: 0.616037,D fake: 0.317736
Epoch[104/1000],d_loss:1.048317,g_loss:1.413687 D real: 0.691332,D fake: 0.369372
Epoch[104/1000],d_loss:0.688330,g_loss:1.629859 D real: 0.728682,D fake: 0.218122
Epoch[104/1000],d_loss:0.990300,g_loss:1.644935 D real: 0.638867,D fake: 0.279301
Epoch[104/1000],d_loss:0.706080,g_loss:1.978722 D real: 0.795352,D fake: 0.284681
Epoch[105/1000],

Epoch[127/1000],d_loss:1.062590,g_loss:1.512857 D real: 0.725419,D fake: 0.429987
Epoch[127/1000],d_loss:1.117925,g_loss:1.424917 D real: 0.618070,D fake: 0.347993
Epoch[127/1000],d_loss:0.991616,g_loss:1.625439 D real: 0.634464,D fake: 0.304396
Epoch[127/1000],d_loss:0.993099,g_loss:1.549413 D real: 0.746610,D fake: 0.387240
Epoch[128/1000],d_loss:0.898192,g_loss:1.590413 D real: 0.684249,D fake: 0.317550
Epoch[128/1000],d_loss:0.860026,g_loss:1.549697 D real: 0.721582,D fake: 0.330069
Epoch[128/1000],d_loss:0.996556,g_loss:1.723084 D real: 0.660128,D fake: 0.325013
Epoch[128/1000],d_loss:1.083815,g_loss:1.220303 D real: 0.630092,D fake: 0.333877
Epoch[129/1000],d_loss:0.852793,g_loss:1.640268 D real: 0.674523,D fake: 0.282568
Epoch[129/1000],d_loss:0.860192,g_loss:1.592161 D real: 0.650874,D fake: 0.253355
Epoch[129/1000],d_loss:0.918555,g_loss:1.461291 D real: 0.676997,D fake: 0.312957
Epoch[129/1000],d_loss:1.017160,g_loss:1.880259 D real: 0.675364,D fake: 0.356030
Epoch[130/1000],

Epoch[152/1000],d_loss:0.920422,g_loss:1.367296 D real: 0.696216,D fake: 0.317462
Epoch[152/1000],d_loss:1.128680,g_loss:1.650403 D real: 0.613617,D fake: 0.348749
Epoch[152/1000],d_loss:0.954087,g_loss:1.368553 D real: 0.707373,D fake: 0.361181
Epoch[152/1000],d_loss:1.108537,g_loss:1.169610 D real: 0.585703,D fake: 0.309668
Epoch[153/1000],d_loss:0.669175,g_loss:1.632725 D real: 0.765771,D fake: 0.260086
Epoch[153/1000],d_loss:1.226076,g_loss:1.365018 D real: 0.689325,D fake: 0.440543
Epoch[153/1000],d_loss:1.005410,g_loss:1.393371 D real: 0.656761,D fake: 0.330808
Epoch[153/1000],d_loss:1.004983,g_loss:1.489738 D real: 0.666414,D fake: 0.344158
Epoch[154/1000],d_loss:0.888842,g_loss:1.540159 D real: 0.708872,D fake: 0.313797
Epoch[154/1000],d_loss:0.878980,g_loss:1.292659 D real: 0.778147,D fake: 0.371417
Epoch[154/1000],d_loss:0.814134,g_loss:1.514006 D real: 0.689953,D fake: 0.274903
Epoch[154/1000],d_loss:1.107716,g_loss:1.742487 D real: 0.610064,D fake: 0.347020
Epoch[155/1000],

Epoch[177/1000],d_loss:0.890980,g_loss:1.734636 D real: 0.709151,D fake: 0.343125
Epoch[177/1000],d_loss:0.915482,g_loss:1.480257 D real: 0.661654,D fake: 0.300656
Epoch[177/1000],d_loss:1.094346,g_loss:1.369233 D real: 0.665952,D fake: 0.376396
Epoch[177/1000],d_loss:0.724077,g_loss:1.644435 D real: 0.737572,D fake: 0.278272
Epoch[178/1000],d_loss:0.998914,g_loss:1.578177 D real: 0.696030,D fake: 0.370101
Epoch[178/1000],d_loss:0.703985,g_loss:1.543093 D real: 0.785735,D fake: 0.298274
Epoch[178/1000],d_loss:0.770997,g_loss:1.511953 D real: 0.721123,D fake: 0.291932
Epoch[178/1000],d_loss:1.058872,g_loss:1.150397 D real: 0.612084,D fake: 0.311411
Epoch[179/1000],d_loss:0.913099,g_loss:1.236809 D real: 0.713949,D fake: 0.352712
Epoch[179/1000],d_loss:0.750928,g_loss:1.509065 D real: 0.760632,D fake: 0.298930
Epoch[179/1000],d_loss:0.993046,g_loss:1.466946 D real: 0.680395,D fake: 0.359799
Epoch[179/1000],d_loss:1.096509,g_loss:1.362053 D real: 0.654661,D fake: 0.374704
Epoch[180/1000],

Epoch[202/1000],d_loss:1.037873,g_loss:1.699888 D real: 0.653716,D fake: 0.324734
Epoch[202/1000],d_loss:0.890054,g_loss:1.359716 D real: 0.703771,D fake: 0.305812
Epoch[202/1000],d_loss:0.898428,g_loss:1.551142 D real: 0.676171,D fake: 0.300440
Epoch[202/1000],d_loss:0.977612,g_loss:1.330540 D real: 0.696273,D fake: 0.361538
Epoch[203/1000],d_loss:0.665669,g_loss:1.427588 D real: 0.834223,D fake: 0.317734
Epoch[203/1000],d_loss:1.635644,g_loss:0.866556 D real: 0.467550,D fake: 0.375004
Epoch[203/1000],d_loss:1.054756,g_loss:1.480376 D real: 0.619309,D fake: 0.320959
Epoch[203/1000],d_loss:0.852059,g_loss:1.610505 D real: 0.794854,D fake: 0.366468
Epoch[204/1000],d_loss:1.001280,g_loss:1.271981 D real: 0.626060,D fake: 0.315767
Epoch[204/1000],d_loss:0.955961,g_loss:1.386537 D real: 0.696540,D fake: 0.354646
Epoch[204/1000],d_loss:0.831386,g_loss:2.094832 D real: 0.706953,D fake: 0.266431
Epoch[204/1000],d_loss:0.830438,g_loss:1.473104 D real: 0.735627,D fake: 0.317644
Epoch[205/1000],

Epoch[227/1000],d_loss:0.873284,g_loss:1.272710 D real: 0.686591,D fake: 0.299906
Epoch[227/1000],d_loss:1.024909,g_loss:1.816762 D real: 0.559454,D fake: 0.239327
Epoch[227/1000],d_loss:0.865683,g_loss:1.408490 D real: 0.723274,D fake: 0.338705
Epoch[227/1000],d_loss:0.995686,g_loss:1.566676 D real: 0.708720,D fake: 0.369853
Epoch[228/1000],d_loss:1.038710,g_loss:1.532632 D real: 0.580583,D fake: 0.272906
Epoch[228/1000],d_loss:1.089031,g_loss:1.284957 D real: 0.612505,D fake: 0.327851
Epoch[228/1000],d_loss:0.856611,g_loss:1.582973 D real: 0.731710,D fake: 0.328510
Epoch[228/1000],d_loss:0.948297,g_loss:1.211462 D real: 0.739900,D fake: 0.387192
Epoch[229/1000],d_loss:1.141567,g_loss:1.610661 D real: 0.596768,D fake: 0.310168
Epoch[229/1000],d_loss:0.936371,g_loss:1.469297 D real: 0.686933,D fake: 0.326460
Epoch[229/1000],d_loss:0.992180,g_loss:1.428647 D real: 0.671894,D fake: 0.357664
Epoch[229/1000],d_loss:1.069287,g_loss:1.433478 D real: 0.603028,D fake: 0.314399
Epoch[230/1000],